<a href="https://colab.research.google.com/github/MarinaWolters/Coding-Tracker/blob/master/W6_Pagerank_Matrix_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!apt update
!apt install gcc python-dev libkrb5-dev
!pip install sparkmagic

In [ ]:
%%capture
%load_ext sparkmagic.magics 

In [ ]:
# TODO: Enter your Master Public DNS with the proper formatting and host

%spark add -s my_session -l python -u http://ec2-54-165-180-197.compute-1.amazonaws.com:8998


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1603284584123_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [ ]:
%%spark

from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry,\
  BlockMatrix
import pandas as pd
from pyspark.sql.functions import lit

initial_graph = pd.DataFrame([{'from': 0, 'to': 1},\
                              {'from': 0, 'to': 2},\
                              {'from': 1, 'to': 2},\
                              {'from': 2, 'to': 0}])

sdf = spark.createDataFrame(initial_graph)

sdf.createOrReplaceTempView('graph')
transfer_weights = spark.sql('''select graph.from as id, 1 / count("to") as weight 
                                from graph group by from''')
weighted_edges = sdf.join(transfer_weights, sdf['from'] == transfer_weights.id)[['from','to','weight']]

# Build matrix M from the weighted edges
M = CoordinateMatrix(weighted_edges.rdd.map(lambda x: MatrixEntry(x[1], x[0], x[2]))).toBlockMatrix()

print (M.toLocalMatrix())

# We'll use all IDs from 0 ... max node ID
n = len(initial_graph[['from']].rename(columns={'from': 'id'}).\
    append(initial_graph[['to']].rename(columns={'to': 'id'})).drop_duplicates()\
    ['id'])

PR_sdf = sdf.select(sdf['from'], lit(1 / n).alias('weight')).distinct()
PR = CoordinateMatrix(PR_sdf.rdd.map(lambda x: MatrixEntry(x[0], 0, x[1]))).toBlockMatrix()
print (PR.toLocalMatrix())

for i in range(15):
  PR = M.multiply(PR)
  print (PR.toLocalMatrix())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseMatrix([[0. , 0. , 1. ],
             [0.5, 0. , 0. ],
             [0.5, 1. , 0. ]])
DenseMatrix([[0.33333333],
             [0.33333333],
             [0.33333333]])
DenseMatrix([[0.33333333],
             [0.16666667],
             [0.5       ]])
DenseMatrix([[0.5       ],
             [0.16666667],
             [0.33333333]])
DenseMatrix([[0.33333333],
             [0.25      ],
             [0.41666667]])
DenseMatrix([[0.41666667],
             [0.16666667],
             [0.41666667]])
DenseMatrix([[0.41666667],
             [0.20833333],
             [0.375     ]])
DenseMatrix([[0.375     ],
             [0.20833333],
             [0.41666667]])
DenseMatrix([[0.41666667],
             [0.1875    ],
             [0.39583333]])
DenseMatrix([[0.39583333],
             [0.20833333],
             [0.39583333]])
DenseMatrix([[0.39583333],
             [0.19791667],
             [0.40625   ]])
DenseMatrix([[0.40625   ],
             [0.19791667],
             [0.39583333]])
DenseMa

In [ ]:
%%spark
weighted_edges.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(from=1, to=2, weight=1.0), Row(from=2, to=0, weight=1.0), Row(from=0, to=1, weight=0.5), Row(from=0, to=2, weight=0.5)]

In [ ]:
#!/bin/bash
sudo pip3 install requests dill nltk scipy scikit-learn pandas==1.0.5 matplotlib networkx seaborn plotly penngrader mypackage1 mypackage2